In [ ]:
import pymysql
import pandas as pd
import passwords

mysqlip = passwords.mysqlip
mysqlpassword = passwords.mysqlpassword
mysqlusername = passwords.mysqlusername
mysqldb = passwords.mysqldb

print(mysqlip)
counties = [
    'lawrence',
    'butler',
    'beaver',
    'allegheny'
]

In [ ]:
def sendSQLCommand(sql):
    try:
        connection = pymysql.connect(host=mysqlip,
                                     user=mysqlusername,
                                     password=mysqlpassword,
                                     db=mysqldb,
                                     charset='utf8mb4',
                                     cursorclass=pymysql.cursors.DictCursor)

        cursor = connection.cursor()
        cursor.execute(sql)
        dockets = cursor.fetchall()
        cursor.close()
        return dockets
    except Exception as e:
        print(e)
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(exc_type, fname, exc_tb.tb_lineno)

In [ ]:
#Charges Eligible for Sealing

#Charges Eligible for Sealing Non-Conviction
print("Charges Eligible Non-Conviction")
sql = '''SELECT county as County,COUNT(*) as `Charges` FROM sealing WHERE DispositionCategory= 'Non-Conviction'
GROUP BY County'''
cases_eligible_nonconviction = sendSQLCommand(sql)
print(cases_eligible_nonconviction)
df_nonconvictions = pd.DataFrame(cases_eligible_nonconviction)
df_nonconvictions['County'] = df_nonconvictions['County'].str.capitalize()
display(df_nonconvictions)
title = 'Charges Eligible for Sealing Because of Non-Conviction'
temp = df_nonconvictions.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',title=title)


#Charges Eligible for sealing under summary conviction
print("Charges Eligible Summary Convivction")
sql = '''SELECT county as County,COUNT(*) as `Charges` FROM sealing WHERE 
ChargeGrade = 'S' and DispositionCategory = 'Conviction' and DispositionDate < '2012-03-27' and (Restitution = 0 or Restitution IS NULL)
GROUP BY County'''
cases_eligible_summary_conviction = sendSQLCommand(sql)
df_summary_convictions = pd.DataFrame(cases_eligible_summary_conviction)
df_summary_convictions['County'] = df_summary_convictions['County'].str.capitalize()
display(df_summary_convictions)
title = 'Charges Eligible for Sealing Because of Summary Conviction'
temp = df_summary_convictions.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',title=title)


#Charges Eligible for sealing under M2,M3, M Eligibilty
print("Charges Eligble M2, M3, and M")
sql = '''SELECT county as County,COUNT(*) as `Charges` FROM sealing WHERE 
ChargeGrade IN ('M2','M3','M') 
and DispositionCategory = 'Conviction' 
and TenYearConvictionFreePeriod = 'True' 
and (Restitution = 0 or Restitution IS NULL)
and ProhibitedConviction IS NULL
and OTNViolation = False
and LifetimeConviction = False
GROUP BY County'''
cases_eligible_m_conviction = sendSQLCommand(sql)
df_misdemeanor_conviction = pd.DataFrame(cases_eligible_m_conviction)
df_misdemeanor_conviction['County'] = df_misdemeanor_conviction['County'].str.capitalize()
display(df_misdemeanor_conviction)
title = 'M2, M3, and M Charges Eligible for Sealing'
temp = df_misdemeanor_conviction.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',title=title)

#Charges Eligible for sealing ALL
print("Charges Eligble All")
sql = '''SELECT county as County,COUNT(*) as `Charges` FROM sealing WHERE 
(DispositionCategory= 'Non-Conviction')
or (ChargeGrade = 'S' and DispositionCategory = 'Conviction' and DispositionDate < '2012-03-27' and (Restitution = 0 or Restitution IS NULL))
or (ChargeGrade IN ('M2','M3','M') 
and DispositionCategory = 'Conviction' 
and TenYearConvictionFreePeriod = 'True' 
and (Restitution = 0 or Restitution IS NULL)
and ProhibitedConviction IS NULL
and OTNViolation = False
and LifetimeConviction = False)
GROUP BY County'''
caseseligible = sendSQLCommand(sql)
df = pd.DataFrame(caseseligible)
df['County'] = df['County'].str.capitalize()
display(df)
title = 'Charges Eligible for Sealing'
temp = df.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',ylabel='Amount of Charges',title=title)
temp.figure.savefig("Sealing Graphs/" + title + ".png",bbox_inches="tight")


#There is probably a better way to add all the sql results but keep their eligibility category
final = [
    {'County':'lawrence'},
    {'County':'butler'},
    {'County':'beaver'},
    {'County':'allegheny'}
]
for item in cases_eligible_nonconviction:
    for x,item2 in enumerate(final):
        if(item['County'] == item2['County']):
            final[x]['Non-Conviction Dispositions'] = item['Charges']

for item in cases_eligible_m_conviction:
    for x,item2 in enumerate(final):
        if(item['County'] == item2['County']):
            final[x]['Eligible Misdemeanor Convictions'] = item['Charges']

for item in cases_eligible_summary_conviction:
    for x,item2 in enumerate(final):
        if(item['County'] == item2['County']):
            final[x]['Eligible Summary Convictions'] = item['Charges']
print(final)

df = pd.DataFrame(final)
df['County'] = df['County'].str.capitalize()
display(df)
title = 'Charges Eligible for Sealing By Eligibilty Category'
temp = df.sort_values(by=['County'],ascending=False).plot(kind="bar",stacked=True,x='County',xlabel='County',ylabel='Amount of Charges',title=title)
temp.figure.savefig("Sealing Graphs/" + title + ".png",bbox_inches="tight")

In [ ]:
#Cases and Charges Currently Eligible for Sealing

final = []
sql = '''WITH temp2 as
(WITH temp AS 
(SELECT *, CASE 
WHEN 
(DispositionCategory= 'Non-Conviction')
or (ChargeGrade = 'S' and DispositionCategory = 'Conviction' and DispositionDate < '2012-03-27' and (Restitution = 0 or Restitution IS NULL))
or (ChargeGrade IN ('M2','M3','M') 
and DispositionCategory = 'Conviction' 
and TenYearConvictionFreePeriod = 'True' 
and (Restitution = 0 or Restitution IS NULL)
and ProhibitedConviction IS NULL
and OTNViolation = False
and LifetimeConviction = False)
THEN 1
ELSE 0
END AS Eligibility
FROM sealing) SELECT *, SUM(Eligibility) as CaseEligibility FROM temp GROUP BY DocketNumber) 
SELECT 
COUNT(IF(CaseEligibility = TotalCharges,1,NULL)) as `Cases Fully Eligible`,
COUNT(IF(CaseEligibility> 0 and CaseEligibility < TotalCharges,1,NULL)) as `Cases Partially Eligible`,
CAST(SUM(CaseEligibility) as UNSIGNED) as `Charges Eligible`,
County 
FROM temp2  GROUP BY County
'''
cases_eligible = sendSQLCommand(sql)

sql = '''WITH temp2 as
(WITH temp AS 
(SELECT *, CASE 
WHEN 
(DispositionCategory= 'Non-Conviction')
or (ChargeGrade = 'S' and DispositionCategory = 'Conviction' and DispositionDate < '2012-03-27' and (Restitution = 0 or Restitution IS NULL))
or (ChargeGrade IN ('M2','M3','M') 
and DispositionCategory = 'Conviction' 
and TenYearConvictionFreePeriod = 'True' 
and (Restitution = 0 or Restitution IS NULL)
and ProhibitedConviction IS NULL
and OTNViolation = False
and LifetimeConviction = False)
THEN 1
ELSE 0
END AS Eligibility
FROM sealing) SELECT *, MIN(Eligibility) as CaseEligibility FROM temp GROUP BY PersonID) 
SELECT COUNT(IF(CaseEligibility = 1,1,NULL)) as `Records Fully Eligible`,
County FROM temp2  GROUP BY County
'''
people_eligible = sendSQLCommand(sql)
print (caseseligible)
final = []
for item in cases_eligible:
    for item2 in people_eligible:
        if(item['County'] == item2['County']):
            temp = item.copy()
            temp['Records Fully Eligible'] = item2['Records Fully Eligible']
            final.append(temp)
            
#print(cases_eligible)
df = pd.DataFrame(final)
df['County'] = df['County'].str.capitalize()
display(df)
title = 'Charges, Cases, and Records Eligible for Sealing'
temp = df.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',ylabel = 'Amount of Cases,Charges, and Records',title=title)
temp.figure.savefig("Sealing Graphs/" + title +".png",bbox_inches="tight")

#Just Lawrence, Beaver, and Butler
df = df[df['County'] != 'Allegheny']
display(df)
title = 'Charges, Cases, and Records Eligible for Sealing \nin Lawrence, Beaver, and Butler County'
temp = df.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County', ylabel = 'Amount of Cases,Charges, and Records',title=title)
temp.figure.savefig("Sealing Graphs/" + title.replace("\n","") +".png",bbox_inches="tight")

In [ ]:
#Seperate way to calculate Fully,Partial,and Eligible Cases
sql = '''WITH temp2 as
(WITH temp AS 
(SELECT *, CASE 
	WHEN 
	(DispositionCategory= 'Non-Conviction')
	or (ChargeGrade = 'S' and DispositionCategory = 'Conviction' and DispositionDate < '2012-03-27' and (Restitution = 0 or Restitution IS NULL))
	or (ChargeGrade IN ('M2','M3','M') 
	and DispositionCategory = 'Conviction' 
	and TenYearConvictionFreePeriod = 'True' 
	and (Restitution = 0 or Restitution IS NULL)
	and ProhibitedConviction IS NULL
	and OTNViolation = False
	and LifetimeConviction = False) THEN 1
		ELSE 0
	END AS Eligibility
FROM sealing) SELECT *, SUM(Eligibility) as CaseEligibility FROM temp GROUP BY DocketNumber) SELECT COUNT(IF(CaseEligibility = TotalCharges,1,NULL)) as Fully_Eligible,
COUNT(IF(CaseEligibility>0 and CaseEligibility <TotalCharges,1,NULL)) as Cases_Partially_Eligible,
COUNT(IF(CaseEligibility=0,1,NULL)) as Cases_Fully_Ineligible,
SUM(CaseEligibility) as Charges_Eligible,
County 
FROM temp2  GROUP BY County'''
caseseligible = sendSQLCommand(sql)
df = pd.DataFrame(caseseligible)
df['County'] = df['County'].str.capitalize()
display(df)
title = 'Cases Partially & Fully Eligible for Sealing'
temp = df.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',title=title)

In [ ]:
#People with Fully Eligible Records
sql = '''WITH temp2 as
(WITH temp AS 
(SELECT *, CASE 
	WHEN 
	(DispositionCategory= 'Non-Conviction')
	or (ChargeGrade = 's' and DispositionCategory = 'Conviction' and DispositionDate < '2012-03-27' and (Restitution = 0 or Restitution IS NULL))
	or (ChargeGrade IN ('M2','M3','M') 
	and DispositionCategory = 'Conviction' 
	and TenYearConvictionFreePeriod = 'True' 
	and (Restitution = 0 or Restitution IS NULL)
	and ProhibitedConviction IS NULL
	and OTNViolation = False
	and LifetimeConviction = False) THEN 1
		ELSE 0
	END AS Eligibility
FROM sealing) SELECT *, MIN(Eligibility) as CaseEligibility FROM temp GROUP BY PersonID) SELECT COUNT(IF(CaseEligibility = 1,1,NULL)) as Eligible,COUNT(IF(CaseEligibility=0,1,NULL)) as Ineligible,COUNT(*) as Total,County FROM temp2  GROUP BY County
'''
caseseligible = sendSQLCommand(sql)
print (caseseligible)
df = pd.DataFrame(caseseligible)
df['County'] = df['County'].str.capitalize()
display(df)
title = 'People with Fully Eligible Records'
temp = df.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',title=title)

In [ ]:
#Charges Eligible for sealing but for 10 Year Conviction
print("Charges Eligble but for 10 year conviction (.2a1)")
sql = '''SELECT county as County,COUNT(*) as `Charges` FROM sealing WHERE 
ChargeGrade IN ('M2','M3','M') 
and DispositionCategory = 'Conviction' 
and TenYearConvictionFreePeriod != 'True' 
and (Restitution = 0 or Restitution IS NULL)
and ProhibitedConviction IS NULL
and OTNViolation = False
and LifetimeConviction = False
GROUP BY County'''
caseseligible = sendSQLCommand(sql)
df = pd.DataFrame(caseseligible)
df['County'] = df['County'].str.capitalize()
display(df)
title = 'Charges Eligible for Sealing But For Ten Year Free Period of Convictions .2a1'
temp = df.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',title=title)

#Charges Eligible in the next 5 years
print("Charges Eligble In the Next 5 years")
sql = '''SELECT county as County,COUNT(*) as `Charges` FROM sealing WHERE 
ChargeGrade IN ('M2','M3','M') 
and DispositionCategory = 'Conviction' 
and TenYearConvictionFreePeriod != 'True' 
and TenYearConvictionFreePeriodEligibleDate < '2027-03-27'
and (Restitution = 0 or Restitution IS NULL)
and ProhibitedConviction IS NULL
and OTNViolation = False
and LifetimeConviction = False
GROUP BY County'''
caseseligible = sendSQLCommand(sql)
df = pd.DataFrame(caseseligible)
df['County'] = df['County'].str.capitalize()
display(df)
title = 'Charges Eligible for Sealing In The Next Five Years'
temp = df.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',title=title)

In [ ]:
#Summary Convictions Eligible In The Next 10 Years
print("Charges Eligible Summary Conviction")
final = []
sql = '''WITH temp2 as
(WITH temp AS 
(SELECT *, CASE 
WHEN 
ChargeGrade = 'S' and DispositionCategory = 'Conviction' and DispositionDate > '2012-03-27' and (Restitution = 0 or Restitution IS NULL)
THEN 1
ELSE 0
END AS Eligibility
FROM sealing) SELECT *, SUM(Eligibility) as CaseEligibility FROM temp GROUP BY DocketNumber) 
SELECT 
COUNT(IF(CaseEligibility = TotalCharges,1,NULL)) as `Cases Fully Eligible`,
COUNT(IF(CaseEligibility> 0 and CaseEligibility < TotalCharges,1,NULL)) as `Cases Partially Eligible`,
CAST(SUM(CaseEligibility) as UNSIGNED) as `Charges Eligible`,
County 
FROM temp2  GROUP BY County
'''
cases_eligible = sendSQLCommand(sql)

sql = '''WITH temp2 as
(WITH temp AS 
(SELECT *, CASE 
WHEN 
ChargeGrade = 'S' and DispositionCategory = 'Conviction' and DispositionDate > '2012-03-27' and (Restitution = 0 or Restitution IS NULL)
THEN 1
ELSE 0
END AS Eligibility
FROM sealing) SELECT *, MIN(Eligibility) as CaseEligibility FROM temp GROUP BY PersonID) 
SELECT COUNT(IF(CaseEligibility = 1,1,NULL)) as `Records Fully Eligible`,
County FROM temp2  GROUP BY County
'''
people_eligible = sendSQLCommand(sql)
print (caseseligible)
final = []
for item in cases_eligible:
    for item2 in people_eligible:
        if(item['County'] == item2['County']):
            temp = item.copy()
            temp['Records Fully Eligible'] = item2['Records Fully Eligible']
            final.append(temp)
            
#print(cases_eligible)
df = pd.DataFrame(final)
df['County'] = df['County'].str.capitalize()
display(df)
title = 'Summary Convictions Eligible for \nSealing in the Next Ten Years'
temp = df.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',ylabel='Amount of Cases, Charges, and Records',title=title)
temp.figure.savefig("Sealing Graphs/" + title.replace("\n","") +".png",bbox_inches="tight")

#Just Lawrence, Beaver, and Butler
df = df[df['County'] != 'Allegheny']
df['County'] = df['County'].str.capitalize()
display(df)
title = 'Summary Convictions Eligible for Sealing in the \nNext Ten Years In Lawrence, Butler, and Beaver Counties'
temp = df.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',ylabel='Amount of Cases, Charges, and Records',title=title)
temp.figure.savefig("Sealing Graphs/" + title.replace("\n","") +".png",bbox_inches="tight")

In [ ]:
#Summary Convictions Eligible In The Next 5 Years
print("Charges Eligible Summary Conviction")
final = []
sql = '''WITH temp2 as
(WITH temp AS 
(SELECT *, CASE 
WHEN 
ChargeGrade = 'S' and DispositionCategory = 'Conviction' and DispositionDate > '2017-03-27' and (Restitution = 0 or Restitution IS NULL)
THEN 1
ELSE 0
END AS Eligibility
FROM sealing) SELECT *, SUM(Eligibility) as CaseEligibility FROM temp GROUP BY DocketNumber) 
SELECT 
COUNT(IF(CaseEligibility = TotalCharges,1,NULL)) as `Cases Fully Eligible`,
COUNT(IF(CaseEligibility> 0 and CaseEligibility < TotalCharges,1,NULL)) as `Cases Partially Eligible`,
CAST(SUM(CaseEligibility) as UNSIGNED) as `Charges Eligible`,
County 
FROM temp2  GROUP BY County
'''
cases_eligible = sendSQLCommand(sql)

sql = '''WITH temp2 as
(WITH temp AS 
(SELECT *, CASE 
WHEN 
ChargeGrade = 'S' and DispositionCategory = 'Conviction' and DispositionDate > '2017-03-27' and (Restitution = 0 or Restitution IS NULL)
THEN 1
ELSE 0
END AS Eligibility
FROM sealing) SELECT *, MIN(Eligibility) as CaseEligibility FROM temp GROUP BY PersonID) 
SELECT COUNT(IF(CaseEligibility = 1,1,NULL)) as `Record Fully Eligible`,
County FROM temp2  GROUP BY County
'''
people_eligible = sendSQLCommand(sql)
print (caseseligible)
final = []
for item in cases_eligible:
    for item2 in people_eligible:
        if(item['County'] == item2['County']):
            temp = item.copy()
            temp['Records Fully Eligible'] = item2['Record Fully Eligible']
            final.append(temp)
            
#print(cases_eligible)
df = pd.DataFrame(final)
df['County'] = df['County'].str.capitalize()
display(df)
title = 'Summary Convictions Eligible for \nSealing in the Next Five Years'
temp = df.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',ylabel='Amount of Cases, Charges, and Records',title=title)
temp.figure.savefig("Sealing Graphs/" + title.replace("\n","") +".png",bbox_inches="tight")

#Just Lawrence, Beaver, and Butler
df = df[df['County'] != 'Allegheny']
display(df)
title = 'Summary Convictions Eligible for Sealing in the \nNext Five Years in Lawrence, Butler, and Beaver Counties'
temp = df.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',ylabel='Amount of Cases, Charges, and Records',title=title)
temp.figure.savefig("Sealing Graphs/" + title.replace("\n","") +".png",bbox_inches="tight")

In [ ]:
#Charges Eligible for sealing but for Prohibited Convictions under 3a1
print("Charges Eligble M2, M3, and M")
sql = '''SELECT county as County,COUNT(*) as `Charges` FROM sealing WHERE 
ChargeGrade IN ('M2','M3','M') 
and DispositionCategory = 'Conviction' 
and TenYearConvictionFreePeriod = 'True' 
and (Restitution = 0 or Restitution IS NULL)
and ProhibitedConviction IS NOT NULL
and OTNViolation = False
and LifetimeConviction = False
GROUP BY County'''
caseseligible = sendSQLCommand(sql)
print(caseseligible)
df = pd.DataFrame(caseseligible)
df['County'] = df['County'].str.capitalize()
display(df)
title = 'Charges Eligible for Sealing But For Prohibited Charges Under .3a1'
temp = df.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',title=title)

In [ ]:
#Prohibited Convicitions Under 3a1
print("Charges Eligible But For 3a1")
final = []
sql = '''WITH temp2 as
(WITH temp AS 
(SELECT *, CASE 
WHEN 
ChargeGrade IN ('M2','M3','M') 
and DispositionCategory = 'Conviction' 
and TenYearConvictionFreePeriod = 'True' 
and (Restitution = 0 or Restitution IS NULL)
and ProhibitedConviction IS NOT NULL
and OTNViolation = False
and LifetimeConviction = False
THEN 1
ELSE 0
END AS Eligibility
FROM sealing) SELECT *, SUM(Eligibility) as CaseEligibility FROM temp GROUP BY DocketNumber) 
SELECT 
COUNT(IF(CaseEligibility = TotalCharges,1,NULL)) as `Cases Fully Eligible`,
COUNT(IF(CaseEligibility> 0 and CaseEligibility < TotalCharges,1,NULL)) as `Cases Partially Eligible`,
CAST(SUM(CaseEligibility) as UNSIGNED) as `Charges Eligible`,
County 
FROM temp2  GROUP BY County
'''
cases_eligible = sendSQLCommand(sql)

sql = '''WITH temp2 as
(WITH temp AS 
(SELECT *, CASE 
WHEN 
ChargeGrade IN ('M2','M3','M') 
and DispositionCategory = 'Conviction' 
and TenYearConvictionFreePeriod = 'True' 
and (Restitution = 0 or Restitution IS NULL)
and ProhibitedConviction IS NOT NULL
and OTNViolation = False
and LifetimeConviction = False
THEN 1
ELSE 0
END AS Eligibility
FROM sealing) SELECT *, MIN(Eligibility) as CaseEligibility FROM temp GROUP BY PersonID) 
SELECT COUNT(IF(CaseEligibility = 1,1,NULL)) as `Record Fully Eligible`,
County FROM temp2  GROUP BY County
'''
people_eligible = sendSQLCommand(sql)
final = []
for item in cases_eligible:
    for item2 in people_eligible:
        if(item['County'] == item2['County']):
            temp = item.copy()
            temp['Records Fully Eligible'] = item2['Record Fully Eligible']
            final.append(temp)
            
#print(cases_eligible)
df = pd.DataFrame(final)
df['County'] = df['County'].str.capitalize()
display(df)
title = 'Cases, Charges, and Records Eligible for \nSealing but for a Prohibited Conviction Under 3a1'
temp = df.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',ylabel='Amount of Cases, Charges, and Records',title=title)
temp.figure.savefig("Sealing Graphs/" + title.replace("\n","") +".png",bbox_inches="tight")


#Just Lawrence, Beaver, and Butler
df = df[df['County'] != 'Allegheny']
display(df)
title = 'Cases, Charges, and Records Eligible for Sealing but for a \nProhibited Conviction Under 3a1 in Lawrence, Butler, & Beaver Counties'
temp = df.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',ylabel='Amount of Cases, Charges, and Records',title=title)
temp.figure.savefig("Sealing Graphs/" + title.replace("\n","") +".png",bbox_inches="tight")

In [ ]:
#Charges/Cases with at least one charge/Full Cases Eligible for sealing but for restitution
print("Charges and Cases Eligible But For Restitution")

sql = '''SELECT *,COUNT(*) as charge_count FROM sealing WHERE 
(ChargeGrade = 'S' and DispositionCategory = 'Conviction' and DispositionDate < '2012-03-27' and Restitution > 0)
OR
(ChargeGrade IN ('M2','M3','M') 
and DispositionCategory = 'Conviction' 
and TenYearConvictionFreePeriod = 'True' 
and Restitution > 0
and ProhibitedConviction IS NULL
and OTNViolation = False
and LifetimeConviction = False)
GROUP BY DocketNumber'''

finalanswer = []

dockets = sendSQLCommand(sql)
charges_buckets = {
    '$100 or Less':0,
    '$250 or Less':0,
    '$500 or Less':0,
    '$1000 or Less':0,
    '$5000 or Less':0,
    "All":0
}
at_least_1_cases_buckets = {
    '$100 or Less':0,
    '$250 or Less':0,
    '$500 or Less':0,
    '$1000 or Less':0,
    '$5000 or Less':0,
    "All":0
}
full_cases_buckets = {
    '$100 or Less':0,
    '$250 or Less':0,
    '$500 or Less':0,
    '$1000 or Less':0,
    '$5000 or Less':0,
    "All":0
}
for docket in dockets:
    if(float(docket['Restitution']) <= 100):
        charges_buckets['$100 or Less'] += docket['charge_count']
        if(docket['charge_count'] == docket['TotalCharges']):
            full_cases_buckets['$100 or Less'] += 1
        else:
            at_least_1_cases_buckets['$100 or Less'] += 1

    if(float(docket['Restitution']) <= 250):
        charges_buckets['$250 or Less'] += docket['charge_count']
        if(docket['charge_count'] == docket['TotalCharges']):
            full_cases_buckets['$250 or Less'] += 1
        else:
            at_least_1_cases_buckets['$250 or Less'] += 1

    if(float(docket['Restitution']) <= 500):
        charges_buckets['$500 or Less'] += docket['charge_count']
        if(docket['charge_count'] == docket['TotalCharges']):
            full_cases_buckets['$500 or Less'] += 1
        else:
            at_least_1_cases_buckets['$500 or Less'] += 1
        
    if(float(docket['Restitution']) <= 1000):
        charges_buckets['$1000 or Less'] += docket['charge_count']
        if(docket['charge_count'] == docket['TotalCharges']):
            full_cases_buckets['$1000 or Less'] += 1
        else:
            at_least_1_cases_buckets['$1000 or Less'] += 1

    
    if(float(docket['Restitution']) <= 5000):
        charges_buckets['$5000 or Less'] += docket['charge_count']
        if(docket['charge_count'] == docket['TotalCharges']):
            full_cases_buckets['$5000 or Less'] += 1
        else:
            at_least_1_cases_buckets['$5000 or Less'] += 1
    
    charges_buckets["All"] += docket['charge_count']
    if(docket['charge_count'] == docket['TotalCharges']):
        full_cases_buckets['All'] += 1
    else:
        at_least_1_cases_buckets["All"] += 1


print(charges_buckets)
print(at_least_1_cases_buckets)
print(full_cases_buckets)

#Get Full Record
#Full Record Eligible But For Fee
sql = '''WITH temp AS 
(SELECT *, CASE 
WHEN 
(ChargeGrade = 'S' and DispositionCategory = 'Conviction' and DispositionDate < '2012-03-27' and Restitution > 0)
OR
(ChargeGrade IN ('M2','M3','M') 
and DispositionCategory = 'Conviction' 
and TenYearConvictionFreePeriod = 'True' 
and Restitution > 0
and ProhibitedConviction IS NULL
and OTNViolation = False
and LifetimeConviction = False)
THEN 1
ELSE 0
END AS Eligibility
FROM sealing) SELECT *, MIN(Eligibility) as CaseEligibility,IF(MIN(Eligibility) = 1,1,NULL) as `Record Fully Eligible` FROM temp GROUP BY PersonID
'''

finalanswer = []

dockets = sendSQLCommand(sql)
full_record_bucket = {
    '$100 or Less':0,
    '$250 or Less':0,
    '$500 or Less':0,
    '$1000 or Less':0,
    '$5000 or Less':0,
    "All":0
}

for docket in dockets:
    if(docket['CaseEligibility'] == 1):
        if(float(docket['Restitution']) <= 100):
            full_record_bucket['$100 or Less'] += 1

        if(float(docket['Restitution']) <= 250):
            full_record_bucket['$250 or Less'] += 1


        if(float(docket['Restitution']) <= 500):
            full_record_bucket['$500 or Less'] += 1

        if(float(docket['Restitution']) <= 1000):
            full_record_bucket['$1000 or Less'] += 1


        if(float(docket['Restitution']) <= 5000):
            full_record_bucket['$5000 or Less'] += 1

        full_record_bucket["All"] += 1






#Combine Both the Cases and Full Record Eligibility
for key,value in at_least_1_cases_buckets.items():
    finalanswer.append({"Amount":key,"Charges Eligible":charges_buckets[key],"Cases Partially Eligible":value,"Full Cases Eligible":full_cases_buckets[key],"Full Records Eligible":full_record_bucket[key]})
    

# for bucket in buckets:
#     finalanswer.append({'Bucket':bucket,'Amount':buckets[bucket]})
print(finalanswer)
df = pd.DataFrame(finalanswer)
#df['Amount'] = df['Amount'].astype(float)
#df['Bucket'] = df['Bucket'].astype(str)
display(df)
#legendarray = ["$100 or less","$250 or less", "$500 or less", "$1000 or less", "All"]
title = 'Charges and Cases Eligible for Sealing \nBut For Restitution Owed'
temp = df.plot(kind="bar",x='Amount',ylabel="Amount of Cases, Charges, and Records",xlabel='Amount Owed',title=title)
temp.figure.savefig("Sealing Graphs/" + title.replace("\n","") +".png",bbox_inches="tight")


In [ ]:
#Full Record Eligible But For Fee
sql = '''WITH temp AS 
(SELECT *, CASE 
WHEN 
(ChargeGrade = 'S' and DispositionCategory = 'Conviction' and DispositionDate < '2012-03-27' and Restitution > 0)
OR
(ChargeGrade IN ('M2','M3','M') 
and DispositionCategory = 'Conviction' 
and TenYearConvictionFreePeriod = 'True' 
and Restitution > 0
and ProhibitedConviction IS NULL
and OTNViolation = False
and LifetimeConviction = False)
THEN 1
ELSE 0
END AS Eligibility
FROM sealing) SELECT *, MIN(Eligibility) as CaseEligibility,IF(MIN(Eligibility) = 1,1,NULL) as `Record Fully Eligible` FROM temp GROUP BY PersonID
'''

finalanswer = []

dockets = sendSQLCommand(sql)
full_record_bucket = {
    '$100 or Less':0,
    '$250 or Less':0,
    '$500 or Less':0,
    '$1000 or Less':0,
    '$5000 or Less':0,
    "All":0
}

for docket in dockets:
    if(docket['CaseEligibility'] == 1):
        if(float(docket['Restitution']) <= 100):
            full_record_bucket['$100 or Less'] += 1

        if(float(docket['Restitution']) <= 250):
            full_record_bucket['$250 or Less'] += 1


        if(float(docket['Restitution']) <= 500):
            full_record_bucket['$500 or Less'] += 1

        if(float(docket['Restitution']) <= 1000):
            full_record_bucket['$1000 or Less'] += 1


        if(float(docket['Restitution']) <= 5000):
            full_record_bucket['$5000 or Less'] += 1

        full_record_bucket["All"] += 1


# print(charges_buckets)
# print(at_least_1_cases_buckets)
# print(full_cases_buckets)
for key,value in full_record_bucket.items():
    finalanswer.append({"Amount":key,"Full Record":full_record_bucket[key]})
    

# for bucket in buckets:
#     finalanswer.append({'Bucket':bucket,'Amount':buckets[bucket]})
print(finalanswer)
df = pd.DataFrame(finalanswer)
#df['Amount'] = df['Amount'].astype(float)
#df['Bucket'] = df['Bucket'].astype(str)
display(df)
#legendarray = ["$100 or less","$250 or less", "$500 or less", "$1000 or less", "All"]
title = 'Full Records Eligible for Sealing But For Restitution Owed Breakdown By Fee (Cumulative)'
temp = df.plot(kind="bar",x='Amount',ylabel="Number of Charges/Cases",title=title)
#temp.figure.savefig("Sealing Graphs/" + title +".png",bbox_inches="tight")


In [ ]:
#Charges and Cases Eligible for Sealing but for an M1 or F Charge in Case (3b)
print("Charges and Cases Eligible for Sealing but for an M1 or F Charge in Case (3b)")
final = []
sql = '''WITH temp2 as
(WITH temp AS 
(SELECT *, CASE 
WHEN 
ChargeGrade IN ('M2','M3','M') 
and DispositionCategory = 'Conviction' 
and TenYearConvictionFreePeriod = 'True' 
and (Restitution = 0 or Restitution IS NULL)
and ProhibitedConviction IS NULL
and OTNViolation = True
and LifetimeConviction = False
THEN 1
ELSE 0
END AS Eligibility
FROM sealing) SELECT *, SUM(Eligibility) as CaseEligibility FROM temp GROUP BY DocketNumber) 
SELECT 
COUNT(IF(CaseEligibility> 0 and CaseEligibility < TotalCharges,1,NULL)) as `Cases Partially Eligible`,
CAST(SUM(CaseEligibility) as UNSIGNED) as `Charges Eligible`,
County 
FROM temp2  GROUP BY County
'''
cases_eligible = sendSQLCommand(sql)


            
#print(cases_eligible)
df = pd.DataFrame(cases_eligible)
df['County'] = df['County'].str.capitalize()
display(df)
title = 'Charges and Cases Eligible for Sealing \nbut for an M1 or F Charge in Case (3b)'
temp = df.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',ylabel='Amount of Charges and Cases',title=title)
temp.figure.savefig("Sealing Graphs/" + title.replace("\n","") +".png",bbox_inches="tight")

#Just Lawrence, Beaver, and Butler
df = df[df['County'] != 'Allegheny']
display(df)
title = 'Charges and Cases Eligible for Sealing but for an M1 or F \nCharge in Case (3b) in Lawrence, Butler, and Beaver Counties'
temp = df.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',ylabel='Amount of Charges and Cases',figsize = (9,5),title=title)
temp.figure.savefig("Sealing Graphs/" + title.replace("\n","") +".png",bbox_inches="tight")

In [ ]:
#Misdemeanor Convictions Eligible for Sealing but for 10 Year Conviction Free Period
print("Charges Eligible Summary Conviction")
final = []
sql = '''WITH temp2 as
(WITH temp AS 
(SELECT *, CASE 
WHEN 
ChargeGrade IN ('M2','M3','M') 
and DispositionCategory = 'Conviction' 
and TenYearConvictionFreePeriod != 'True' 
and (Restitution = 0 or Restitution IS NULL)
and ProhibitedConviction IS NULL
and OTNViolation = False
and LifetimeConviction = False
THEN 1
ELSE 0
END AS Eligibility
FROM sealing) SELECT *, SUM(Eligibility) as CaseEligibility FROM temp GROUP BY DocketNumber) 
SELECT 
COUNT(IF(CaseEligibility = TotalCharges,1,NULL)) as `Cases Fully Eligible`,
COUNT(IF(CaseEligibility> 0 and CaseEligibility < TotalCharges,1,NULL)) as `Cases Partially Eligible`,
CAST(SUM(CaseEligibility) as UNSIGNED) as `Charges Eligible`,
County 
FROM temp2  GROUP BY County
'''
cases_eligible = sendSQLCommand(sql)

sql = '''WITH temp2 as
(WITH temp AS 
(SELECT *, CASE 
WHEN 
ChargeGrade IN ('M2','M3','M') 
and DispositionCategory = 'Conviction' 
and TenYearConvictionFreePeriod != 'True' 
and (Restitution = 0 or Restitution IS NULL)
and ProhibitedConviction IS NULL
and OTNViolation = False
and LifetimeConviction = False
THEN 1
ELSE 0
END AS Eligibility
FROM sealing) SELECT *, MIN(Eligibility) as CaseEligibility FROM temp GROUP BY PersonID) 
SELECT COUNT(IF(CaseEligibility = 1,1,NULL)) as `Record Fully Eligible`,
County FROM temp2  GROUP BY County
'''
people_eligible = sendSQLCommand(sql)
print (caseseligible)
final = []
for item in cases_eligible:
    for item2 in people_eligible:
        if(item['County'] == item2['County']):
            temp = item.copy()
            temp['Records Fully Eligible'] = item2['Record Fully Eligible']
            final.append(temp)
            
#print(cases_eligible)
df = pd.DataFrame(final)
df['County'] = df['County'].str.capitalize()
display(df)
title = 'Misdemeanor Convictions Eligible for Sealing \nbut for Ten Year Conviction Free Period Under 2a1'
temp = df.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',ylabel='Amount of Cases, Charges, and Records',title=title)
temp.figure.savefig("Sealing Graphs/" + title.replace("\n","") +".png",bbox_inches="tight")


#Just Lawrence, Beaver, and Butler
df = df[df['County'] != 'Allegheny']
display(df)
title = 'Misdemeanor Convictions Eligible for Sealing \nbut for Ten Year Conviction Free Period Under 2a1 for \nin Lawrence, Butler and Beaver Counties'
temp = df.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',ylabel='Amount of Cases, Charges, and Records',figsize = (9,5),title=title)
temp.figure.savefig("Sealing Graphs/" + title.replace("\n","") +".png",bbox_inches="tight")

In [ ]:
#Misdemeanor Convictions Eligible for \nSealing if 2a1 was Reduced to Five Years
print("Misdemeanor Convictions Eligible for Sealing if 2a1 was Reduced to Five Years")
final = []
sql = '''WITH temp2 as
(WITH temp AS 
(SELECT *, CASE 
WHEN 
ChargeGrade IN ('M2','M3','M') 
and DispositionCategory = 'Conviction' 
and TenYearConvictionFreePeriod != 'True' 
and TenYearConvictionFreePeriodEligibleDate BETWEEN '2022-03-27' and '2027-03-27'
and (Restitution = 0 or Restitution IS NULL)
and ProhibitedConviction IS NULL
and OTNViolation = False
and LifetimeConviction = False
THEN 1
ELSE 0
END AS Eligibility
FROM sealing) SELECT *, SUM(Eligibility) as CaseEligibility FROM temp GROUP BY DocketNumber) 
SELECT 
COUNT(IF(CaseEligibility = TotalCharges,1,NULL)) as `Cases Fully Eligible`,
COUNT(IF(CaseEligibility> 0 and CaseEligibility < TotalCharges,1,NULL)) as `Cases Partially Eligible`,
CAST(SUM(CaseEligibility) as UNSIGNED) as `Charges Eligible`,
County 
FROM temp2  GROUP BY County
'''
cases_eligible = sendSQLCommand(sql)

sql = '''WITH temp2 as
(WITH temp AS 
(SELECT *, CASE 
WHEN 
ChargeGrade IN ('M2','M3','M') 
and DispositionCategory = 'Conviction' 
and TenYearConvictionFreePeriod != 'True' 
and TenYearConvictionFreePeriodEligibleDate BETWEEN '2022-03-27' and '2027-03-27'
and (Restitution = 0 or Restitution IS NULL)
and ProhibitedConviction IS NULL
and OTNViolation = False
and LifetimeConviction = False
THEN 1
ELSE 0
END AS Eligibility
FROM sealing) SELECT *, MIN(Eligibility) as CaseEligibility FROM temp GROUP BY PersonID) 
SELECT COUNT(IF(CaseEligibility = 1,1,NULL)) as `Record Fully Eligible`,
County FROM temp2  GROUP BY County
'''
people_eligible = sendSQLCommand(sql)
print (caseseligible)
final = []
for item in cases_eligible:
    for item2 in people_eligible:
        if(item['County'] == item2['County']):
            temp = item.copy()
            temp['Records Fully Eligible'] = item2['Record Fully Eligible']
            final.append(temp)
            
#print(cases_eligible)
df = pd.DataFrame(final)
df['County'] = df['County'].str.capitalize()
display(df)
title = 'Misdemeanor Convictions Eligible for \nSealing if 2a1 was Reduced to Five Years'
temp = df.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',ylabel='Amount of Cases, Charges, and Records',title=title)
temp.figure.savefig("Sealing Graphs/" + title.replace("\n","") +".png",bbox_inches="tight")


#Just Lawrence, Beaver, and Butler
df = df[df['County'] != 'Allegheny']
display(df)
title = 'Misdemeanor Convictions Eligible for \nSealing if 2a1 was Reduced to Five Years \n in Lawrence, Butler, and Beaver Counties'
temp = df.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',ylabel='Amount of Cases, Charges, and Records',title=title)
temp.figure.savefig("Sealing Graphs/" + title.replace("\n","") +".png",bbox_inches="tight")

In [ ]:
#Lifetime Conviction
print("Charges Eligible But for Lifetime Conviction")
final = []
sql = '''WITH temp2 as
(WITH temp AS 
(SELECT *, CASE 
WHEN 
ChargeGrade IN ('M2','M3','M') 
and DispositionCategory = 'Conviction' 
and TenYearConvictionFreePeriod = 'True' 
and (Restitution = 0 or Restitution IS NULL)
and ProhibitedConviction IS NULL
and OTNViolation = False
and LifetimeConviction = True
THEN 1
ELSE 0
END AS Eligibility
FROM sealing) SELECT *, SUM(Eligibility) as CaseEligibility FROM temp GROUP BY DocketNumber) 
SELECT 
COUNT(IF(CaseEligibility = TotalCharges,1,NULL)) as `Cases Fully Eligible`,
COUNT(IF(CaseEligibility> 0 and CaseEligibility < TotalCharges,1,NULL)) as `Cases Partially Eligible`,
CAST(SUM(CaseEligibility) as UNSIGNED) as `Charges Eligible`,
County 
FROM temp2  GROUP BY County
'''
cases_eligible = sendSQLCommand(sql)

sql = '''WITH temp2 as
(WITH temp AS 
(SELECT *, CASE 
WHEN 
ChargeGrade IN ('M2','M3','M') 
and DispositionCategory = 'Conviction' 
and TenYearConvictionFreePeriod = 'True' 
and (Restitution = 0 or Restitution IS NULL)
and ProhibitedConviction IS NULL
and OTNViolation = False
and LifetimeConviction = True
THEN 1
ELSE 0
END AS Eligibility
FROM sealing) SELECT *, MIN(Eligibility) as CaseEligibility FROM temp GROUP BY PersonID) 
SELECT COUNT(IF(CaseEligibility = 1,1,NULL)) as `Record Fully Eligible`,
County FROM temp2  GROUP BY County
'''
people_eligible = sendSQLCommand(sql)
print (caseseligible)
final = []
for item in cases_eligible:
    for item2 in people_eligible:
        if(item['County'] == item2['County']):
            temp = item.copy()
            temp['Records Fully Eligible'] = item2['Record Fully Eligible']
            final.append(temp)
            
#print(cases_eligible)
df = pd.DataFrame(final)
df['County'] = df['County'].str.capitalize()
display(df)
title = 'Cases, Charges, and Records Eligible \nfor Sealing But For Lifetime Conviction'
temp = df.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',ylabel='Amount of Cases, Charges, and Records',title=title)
temp.figure.savefig("Sealing Graphs/" + title.replace("\n","") +".png",bbox_inches="tight")

#Just Lawrence, Beaver, and Butler
df = df[df['County'] != 'Allegheny']
display(df)
title = 'Cases, Charges, and Records Eligible for Sealing But For Lifetime \nConviction in Lawrence, Butler, and Beaver Counties'
temp = df.sort_values(by=['County'],ascending=False).plot(kind="bar",x='County',ylabel='Amount of Cases, Charges, and Records',title=title)
temp.figure.savefig("Sealing Graphs/" + title.replace("\n","") +".png",bbox_inches="tight")

In [ ]:
#Lifetime Conviction Specific Crime Breakdown
import json 

final = []
sql = '''SELECT * FROM sealing WHERE
ChargeGrade IN ('M2','M3','M') 
and DispositionCategory = 'Conviction' 
and TenYearConvictionFreePeriod = 'True' 
and (Restitution = 0 or Restitution IS NULL)
and ProhibitedConviction IS NULL
and OTNViolation = False
and LifetimeConviction = True
GROUP BY DocketNumber
'''
specific_convictions = {"indecent_exposure_3127": 0, "sex_with_animal_3129": 0, "fail_register_sex_offender_4915": 0, "inmate_weapon_5122": 0, "paramilitary_training_5515": 0, "abuse_of_a_corpse_5510": 0}
cases_eligible = sendSQLCommand(sql)
#print(cases_eligible)
for docket in dockets:
    #print(docket['LifetimeConvictionArray'])
    lifetime_dict = json.loads(docket['LifetimeConvictionArray'])
    for key in specific_convictions:
        if(lifetime_dict[key] == True):
            specific_convictions[key] += 1

print(specific_convictions)
temp = {}

for item in specific_convictions:
    if(item == 'sex_with_animal_3129'):
        final.append({'Crime':'Beastiality','Cases':specific_convictions[item]})
    else:
        final.append({'Crime':item.replace("_"," ").title()[0:-4],'Cases':specific_convictions[item]})
df = pd.DataFrame(final)
display(df)
title = 'Cases Blocked By Specific Lifetime Convictions'
temp = df.sort_values(by=['Crime'],ascending=False).plot(kind="bar",x='Crime',ylabel='Amount of Cases',title=title)
temp.figure.savefig("Sealing Graphs/" + title.replace("\n","") +".png",bbox_inches="tight")

In [ ]:
#Eligible Non Conviction Breakdown by Filing Date
sql = '''
SELECT FilingDate,SUM(Total) as Dispositions FROM (
SELECT YEAR(filingDate) as FilingDate,count(*) as Total FROM pennsylvania2022.allegheny_charges as charges INNER JOIN pennsylvania2022.allegheny_cases as cases on charges.docketNumber = cases.docketNumber 
WHERE status in ('Closed','Adjudicated','Adjudicated/Closed') and (disposition LIKE '%nolle%' or disposition like "%dismiss%" or disposition like '%withdraw%' or disposition like 'not guilty%') group by FilingDate
UNION
SELECT YEAR(filingDate) as FilingDate,count(*) as Total FROM pennsylvania2022.butler_charges as charges INNER JOIN pennsylvania2022.butler_cases as cases on charges.docketNumber = cases.docketNumber 
WHERE status in ('Closed','Adjudicated','Adjudicated/Closed')  and (disposition LIKE '%nolle%' or disposition like "%dismiss%" or disposition like '%withdraw%' or disposition like 'not guilty%') group by FilingDate
UNION
SELECT YEAR(filingDate) as FilingDate,count(*) as Total FROM pennsylvania2022.beaver_charges as charges INNER JOIN pennsylvania2022.beaver_cases as cases on charges.docketNumber = cases.docketNumber 
WHERE status in ('Closed','Adjudicated','Adjudicated/Closed') and (disposition LIKE '%nolle%' or disposition like "%dismiss%" or disposition like '%withdraw%' or disposition like 'not guilty%') group by FilingDate
UNION
SELECT YEAR(filingDate) as FilingDate,count(*) as Total FROM pennsylvania2022.lawrence_charges as charges INNER JOIN pennsylvania2022.lawrence_cases as cases on charges.docketNumber = cases.docketNumber 
WHERE status in ('Closed','Adjudicated','Adjudicated/Closed') and (disposition LIKE '%nolle%' or disposition like "%dismiss%" or disposition like '%withdraw%' or disposition like 'not guilty%') group by FilingDate
) as temp GROUP BY FilingDate ORDER BY FilingDate DESC
'''
dispositions = sendSQLCommand(sql)
df = pd.DataFrame(dispositions)

print(df.dtypes)
#df['Dispositions'].astype(float)
df["Dispositions"] = pd.to_numeric(df["Dispositions"])
print(df.dtypes)
display(df)
title = 'Non-Conviction Dispositions Eligible for Sealing in \n Closed Common Pleas Criminal Case In All Counties By Filing Date'
temp = df.sort_values(by=['FilingDate'],ascending=True).plot(kind="bar",x='FilingDate',xlabel = 'Filing Date',ylabel="Number of Dispositions",title=title)
temp.figure.savefig("Sealing Graphs/" + title.replace("\n","") +".png",bbox_inches="tight")